In [14]:
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load DB URL
load_dotenv()
DB_URL = os.getenv("DB_URL")
engine = create_engine(DB_URL)

In [17]:
load_dotenv(override=True)

True

In [15]:
def fetch_stock_data(ticker, period="5d"):
    data = yf.download(ticker, period=period)
    data.reset_index(inplace=True)
    data.rename(columns={
        'Date': 'date',
        'Open': 'open_price',
        'High': 'high',
        'Low': 'low',
        'Close': 'close_price',
        'Volume': 'volume'
    }, inplace=True)
    data['ticker'] = ticker
    return data[['ticker', 'date', 'open_price', 'high', 'low', 'close_price', 'volume']]


def save_to_db(df, table_name):
    df.to_sql(table_name, engine, if_exists='append', index=False)



In [16]:
# Test run
if __name__ == "__main__":
    df = fetch_stock_data("AAPL")
    save_to_db(df, "stock_data")
    print("Stock data saved successfully!")


C:\Users\Admin\AppData\Local\Temp\ipykernel_7804\4000630198.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=period)
[*********************100%***********************]  1 of 1 completed


InterfaceError: (pyodbc.InterfaceError) ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/20/rvf5)

In [18]:
import pyodbc
print(pyodbc.drivers())


['SQL Server', 'ODBC Driver 17 for SQL Server', 'SQL Server Native Client RDA 11.0', 'SQL Server Native Client 11.0', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']


In [19]:
from sqlalchemy import create_engine

engine = create_engine(DB_URL)
try:
    with engine.connect() as conn:
        print("Connection successful!")
except Exception as e:
    print(e)


(pyodbc.InterfaceError) ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/20/rvf5)
